## Convert files from Thermo

ThermoRawFileParser https://github.com/compomics/ThermoRawFileParser

Convert profile MS1 data from Thermo raw files to centroid (MS1 and MS2) mzml
Move all the files to data/raw directory and you can use the following script to generate a samples.tsv file automatically:

In [13]:
!(cd data/raw && ls *.raw > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/raw/filelist.txt", names=header_list)
df["sample_name"]=df["sample_name"].replace(".raw", value="", regex=True)
df["comment"] = ""
df["MAPnumber"] = ""
df.to_csv("config/samples.tsv", sep="\t")
df

,sample_name,comment,MAPnumber
0,20210827_UMETAB219_POS_ISP2_Plate-2_MDNA_WGS_1...,,


## Import libraries

In [2]:
from pyopenms import *
from pandas import DataFrame
import pandas as pd
import pyteomics
from pyteomics.openms import featurexml
import numpy as np
import sys
from pyteomics import mztab

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


## Preprocessing step

The first preprocessing function consists of six steps:

### 1) PrecursorCorrection (To the "highest intensity MS1 peak")

The highestintensity is used directly right after the file introduction, in order to correct any wrong MS1 precursor annotation - this means it will correct in the precursor window (e.g. 0.01 Da) and then use the highest intensity peak as the precursor. This is to correct for any instrument error. Since we assume that in the given mass window, the precursor with the hightest intensity was actually fragmented instead of any other (top-n method). This method is by default used in the Thermo Orbitrap instrument we have.

openMS definition:

"Selection of the peak with the highest intensity as corrected precursor mass in a given mass range (e.g. precursor mass +/- 0.2 Da)

For each MS2 spectrum the corresponding MS1 spectrum is determined by using the rt information of the precursor. In the MS1, the peak with the highest intensity in a given mass range to the uncorrected precursor m/z is selected and used as corrected precursor m/z."



In [3]:
example_filename= "results/Epemicins/interim/Epemicins.mzML"
exp = MSExperiment()
MzMLFile().load(example_filename, exp)
exp.sortSpectra(True)
    
delta_mzs= []
mzs = []
rts= []
PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
MzMLFile().store("results/Epemicins/interim/precursorcorrected_Epemicins.mzML", exp)

### 2) Mass trace detection

A mass trace extraction method that gathers peaks similar in m/z and moving along retention time.

Peaks of a MSExperiment are sorted by their intensity and stored in a list of potential chromatographic apex positions. Only peaks that are above the noise threshold (user-defined) are analyzed and only peaks that are n times above this minimal threshold are considered as apices. This saves computational resources and decreases the noise in the resulting output.

Starting with these, mass traces are extended in- and decreasingly in retention time. During this extension phase, the centroid m/z is computed on-line as an intensity-weighted mean of peaks.

The extension phase ends when either the frequency of gathered peaks drops below a threshold (min_sample_rate, see MassTraceDetection parameters) or when the number of missed scans exceeds a threshold (trace_termination_outliers, see MassTraceDetection parameters).

Finally, only mass traces that pass a filter (a certain minimal and maximal length as well as having the minimal sample rate criterion fulfilled) get added to the result.

In [4]:
mass_traces = []
mtd = MassTraceDetection()
mtd_par = mtd.getDefaults()
mtd_par.setValue("mass_error_ppm", 10.0) 
mtd_par.setValue("noise_threshold_int", 1.0e04)
mtd.setParameters(mtd_par)
mtd.run(exp, mass_traces, 0)

Progress of 'mass trace detection':
-- done [took 0.08 s (CPU), 0.08 s (Wall)] -- 


### 3) Elution peak detection

Extracts chromatographic peaks from a mass trace.

Mass traces may consist of several consecutively (partly overlapping) eluting peaks, e.g., stemming from (almost) isobaric compounds that are separated by retention time. Especially in metabolomics, isomeric compounds with exactly the same mass but different retentional behaviour may still be contained in the same mass trace.

This method first applies smoothing on the mass trace's intensities, then detects local minima/maxima in order to separate the chromatographic peaks from each other. Detection of maxima is performed on the smoothed intensities and uses a fixed peak width (given as parameter chrom_fwhm) within which only a single maximum is expected. Currently smoothing is done using SavitzkyGolay smoothing with a second order polynomial and a frame length of the fixed peak width.

Depending on the "width_filtering" parameters, mass traces are filtered by length in seconds ("fixed" filter) or by quantile.

The output of the algorithm is a set of chromatographic peaks for each mass trace, i.e. a vector of split mass traces (see ElutionPeakDetection parameters).

In general, a user would want to call the "detectPeaks" functions, potentially followed by the "filterByPeakWidth" function.

This method in other words is "deconvolution".

In [5]:
mass_traces_split = []
mass_traces_final = []
epd = ElutionPeakDetection()
epd_par = epd.getDefaults()
epd_par.setValue("width_filtering", "fixed")
epd.setParameters(epd_par)
epd.detectPeaks(mass_traces, mass_traces_split)
    
if (epd.getParameters().getValue("width_filtering") == "auto"):
     epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
else:
    mass_traces_final = mass_traces_split

Progress of 'elution peak detection':
-- done [took 0.77 s (CPU), 0.16 s (Wall)] -- 


### 4) Feature detection

FeatureFinderMetabo assembles metabolite features from singleton mass traces.

Mass traces alone would allow for further analysis such as metabolite ID or statistical evaluation. However, in general, monoisotopic mass traces are accompanied by satellite C13 peaks and thus may render the analysis more difficult. FeatureFinderMetabo fulfills a further data reduction step by assembling compatible mass traces to metabolite features (that is, all mass traces originating from one metabolite). To this end, multiple metabolite hypotheses are formulated and scored according to how well differences in RT (optional), m/z or intensity ratios match to those of theoretical isotope patterns.

If the raw data scans contain the scan polarity information, it is stored as meta value "scan_polarity" in the output file.

Mass trace clustering can be done using either 13C distances or a linear model (Kenar et al) – see parameter 'ffm:mz_scoring_13C'. Generally, for lipidomics, use 13C, since lipids contain a lot of 13C. For general metabolites, the linear model is usually more appropriate. To decide what is better, the total number of features can be used as indirect measure.

the lower(!) the better (since more mass traces are assembled into single features). Detailed information is stored in the featureXML output: it contains meta-values for each feature about the mass trace differences (inspectable via TOPPView). 

By default, the linear model is used.

In [13]:
feature_map_FFM = FeatureMap()
feat_chrom = []
ffm = FeatureFindingMetabo()
ffm_par = ffm.getDefaults() 
ffm_par.setValue("isotope_filtering_model", "none")
ffm_par.setValue("remove_single_traces", "false")
ffm_par.setValue("mz_scoring_by_elements", "false")
ffm_par.setValue("report_convex_hulls", "true")
ffm.setParameters(ffm_par)
ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
feature_map_FFM.setUniqueIds()
fh = FeatureXMLFile()
filename= "results/Epemicins/interim/precursorcorrected_Epemicins.mzML"
feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
fh.store('./results/Epemicins/interim/FFM_Epemicins.featureXML', feature_map_FFM)

Progress of 'assembling mass traces to features':
-- done [took 0.63 s (CPU), 0.14 s (Wall)] -- 


### 5) Metabolite adduct decharger (MetaboliteFeatureDeconvolution)

For each peak, this algorithm reconstructs neutral masses by enumerating all possible adducts with matching charge. 
You can add the list of adduct files and database files for the algorithm to parse through.
With SIRIUS, an algorithm that is later used, you are only able to use singly charged adducts so charges higher than 1 are filtered out.

In [14]:
mfd = MetaboliteFeatureDeconvolution()
mdf_par = mfd.getDefaults()
mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
mdf_par.setValue("charge_min", 1, "Minimal possible charge")
mdf_par.setValue("charge_max", 1, "Maximal possible charge")
mdf_par.setValue("charge_span_max", 1)
mdf_par.setValue("max_neutrals", 1)
mfd.setParameters(mdf_par)
    
feature_map_DEC = FeatureMap()
cons_map0 = ConsensusMap()
cons_map1 = ConsensusMap()
mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
fxml = FeatureXMLFile()
fh.store('./results/Epemicins/interim/MFD_Epemicins.featureXML', feature_map_DEC)

Compomer: Da -41.0003; q_net 0; logP -3.91202[[ (H2O1Na1) --> () ]]
 Compomer: Da -36.0449; q_net 0; logP -4.60517[[ (H2O1H4N1) --> () ]]
 Compomer: Da -21.9819; q_net 0; logP -2.12026[[ (Na1) --> (H1) ]]
 Compomer: Da -19.0184; q_net 0; logP -2.81341[[ (H1H2O1) --> () ]]
 Compomer: Da -17.0265; q_net 0; logP -2.81341[[ (H4N1) --> (H1) ]]
 Compomer: Da -4.9554; q_net 0; logP -3.91202[[ (Na1) --> (H4N1) ]]
 Compomer: Da 4.9554; q_net 0; logP -3.91202[[ (H4N1) --> (Na1) ]]
 Compomer: Da 17.0265; q_net 0; logP -2.81341[[ (H1) --> (H4N1) ]]
 Compomer: Da 19.0184; q_net 0; logP -2.81341[[ () --> (H1H2O1) ]]
 Compomer: Da 21.9819; q_net 0; logP -2.12026[[ (H1) --> (Na1) ]]
 Compomer: Da 36.0449; q_net 0; logP -4.60517[[ () --> (H2O1H4N1) ]]
 Compomer: Da 41.0003; q_net 0; logP -3.91202[[ () --> (H2O1Na1) ]]
 

MassExplainer table size: 12
Generating Masses with threshold: -6.90776 ...
done
0 of 1629 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 1629 to 1629
Found 1629 putative edges (of 66430) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
<Using solver 'coinor' ...> occurred 5 times
 Branch and cut took 0.224033 seconds,  with objective value: 27.82.
<Optimal solution found!> occurred 5 times
ILP score is: 27.82
Agreeing charges: 196/1500


In [15]:
with featurexml.read('./results/Epemicins/interim/MFD_Epemicins.featureXML') as f:
    features_list = [FXML for FXML in f]
    
df = pd.DataFrame() 
for feat in features_list:
    idx = feat['id']
    for key in feat.keys():
        if key == 'id':
           pass
        elif key == 'position':
            pos_list = feat['position']
            for pos in pos_list:
                if pos['dim'] == '0':
                    df.loc[idx, 'RT'] = pos['position']
                elif pos['dim'] == '1':
                    df.loc[idx, 'mz'] = pos['position']
        elif key == 'intensity':
            df.loc[idx, 'intensity'] = feat['intensity']
        elif key == 'charge':
            df.loc[idx, 'charge'] = feat['charge']
        elif key == 'FWHM':
            df.loc[idx, 'FWHM'] = feat['FWHM']
        elif key == 'max_height':
            df.loc[idx, 'max_height'] = feat['max_height']
        elif key == 'num_of_masstraces':
            df.loc[idx, 'num_of_masstraces'] = feat['num_of_masstraces']
        elif key == 'masstrace_intesity':
            df.loc[idx, 'masstrace_intesity'] = feat['masstrace_intesity']
        elif key == 'masstrace_centroid_rt':
            df.loc[idx, 'masstrace_centroid_rt'] = feat['masstrace_centroid_rt']
        elif key == 'masstrace_centroid_mz':
            df.loc[idx, 'masstrace_centroid_mz'] = feat['masstrace_centroid_mz']
        elif key == 'isotope_distances':
            df.loc[idx, 'isotope_distances'] = feat['isotope_distances']
        elif key == "adducts":
            if 'adducts' not in df.columns:
                df['adducts'] = ''
                df.loc[idx, "adducts"] = feat["adducts"]
            else:
                df.loc[idx, "adducts"] = feat["adducts"]
df= df[df["num_of_masstraces"]>=2.0]

In [12]:
df=df.reset_index()
df= df.rename(columns={"index": "FeatureID"})
df= df.set_index("FeatureID")
df.to_csv("results/Epemicins/features_Epemicins.csv")
df

,identifier,chemical_formula,smiles,inchi_key,description,exp_mass_to_charge,calc_mass_to_charge,charge,retention_time,taxid,...,opt_global_rank,opt_global_explainedPeaks,opt_global_explainedIntensity,opt_global_median_mass_error_fragment_peaks_ppm,opt_global_median_absolute_mass_error_fragment_peaks_ppm,opt_global_mass_error_precursor_ppm,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id
FeatureID,,,,,,,,,,,,,,,,,,,,,
0,None,C25H45NO5,None,None,None,457.363953,None,None,327.701974,None,...,1,20,0.858468,-1.558118,1.613195,0.773572,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572
1,None,C21H41N7O3,None,None,None,457.363953,None,None,327.701974,None,...,2,19,0.857305,-1.582289,1.644100,6.644975,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572
2,None,C23H43N4O4,None,None,None,457.363953,None,None,327.701974,None,...,3,20,0.858468,-1.558118,1.678539,3.709274,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572
3,None,C22H47O8,None,None,None,457.363953,None,None,327.701974,None,...,4,19,0.857305,-1.582289,1.644100,6.633404,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572
4,None,C28H43N2O2,None,None,None,457.363953,None,None,327.701974,None,...,5,18,0.845485,-1.716469,1.716469,-5.086260,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8451,None,C12H28N6O3S,None,None,None,337.199054,None,None,465.541832,None,...,6,17,0.656215,-2.445109,2.908630,-7.657475,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337
8452,None,C20H24N4O,None,None,None,337.199054,None,None,465.541832,None,...,7,14,0.380458,-1.762479,1.762479,-9.589960,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337
8453,None,C13H22N9O2,None,None,None,337.199054,None,None,465.541832,None,...,8,14,0.563654,-1.058188,2.252102,6.321840,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337


### 6) PrecursorCorrection (To the "nearest feature”)

This algorithm is used after feature detection, adduct grouping and even identification via accurate mass search. It basically allows the precursor correction on MS2 level. 

Which means that if there are MS2 spectra in my feature space which have been measured in isotope traces, it “corrects” the MS2 spectrum annotation to the monoisotopic trace. That is why you have a high mass deviation 100 pm, but 0.0 rt tolerance. So it basically corrects the MS2 to the feature centroid that can be found/mapped by SIRIUS::preprocessing later on.

Wrong assignment of the mono-isotopic mass for precursors are assumed:

1. if precursor_mz matches the mz of a non-monoisotopic feature mass trace and 
2. in the case that believe_charge is true: if feature_charge matches the precursor_charge In the case of wrong mono-isotopic assignment several options for correction are available: keep_original will create a copy of the precursor and tandem spectrum for the new mono-isotopic mass trace and retain the original one. all_matching_features does this not for only the closest feature but all features in a question.

In [21]:
PrecursorCorrection.correctToNearestFeature(feature_map_DEC, exp, 0.0, 100.0, True, False, False, False, 3, 0)

set()

HighResPrecursorMassCorrector warning: at least one feature has no convex hull - omitting feature for matching


### SIRIUS Adapter
The SIRIUS function is optional and includes the SIRIUS Adapter Algorithm from the Boecher lab. 


The algorithm generates formula prediction from scores calculated from 1) MS2 fragmentation scores (ppm error + intensity) and 2) MS1 isotopic pattern scores.


It can only compute feautures that are singly charged. There is also a timeout for compounds (compound timeout so that it doesn't compute for longer than 100 seconds per feature, which normally happens with larger molecules).


-sirius:compound_timeout <number>                    
Maximal computation time in seconds for a single compound. 0 for an infinite amount of time. (default: '100' min: '0')

    
This algorith can help in data dereplication and analysis for direct library search. 

In [ ]:
sirius_algo = SiriusAdapterAlgorithm()
sirius_algo_par = sirius_algo.getDefaults()
sirius_algo_par.setValue("preprocessing:filter_by_num_masstraces", 2) 
sirius_algo_par.setValue("preprocessing:precursor_mz_tolerance", 10.0) #default
sirius_algo_par.setValue("preprocessing:precursor_mz_tolerance_unit", "ppm")
sirius_algo_par.setValue("preprocessing:precursor_rt_tolerance", 5.0) #default
sirius_algo_par.setValue("preprocessing:feature_only", "true")
sirius_algo_par.setValue("sirius:profile", "orbitrap")
sirius_algo_par.setValue("sirius:db", "none")
sirius_algo_par.setValue("sirius:ions_considered", "[M+H]+, [M-H2O+H]+, [M+Na]+, [M+NH4]+")
sirius_algo_par.setValue("sirius:candidates", 10)
sirius_algo_par.setValue("sirius:elements_enforced", "CHN[15]OS") 
sirius_algo_par.setValue("project:processors", 2)
sirius_algo_par.setValue("fingerid:db", "BIO")
sirius_algo.setParameters(sirius_algo_par)
    
featureinfo = "./results/Epemicins/interim/MFD_Epemicins.featureXML"
fm_info = FeatureMapping_FeatureMappingInfo()
feature_mapping = FeatureMapping_FeatureToMs2Indices() 
sirius_algo.preprocessingSirius(featureinfo,
                                exp,
                                fm_info,
                                feature_mapping)
sirius_algo.logFeatureSpectraNumber(featureinfo, 
                                    feature_mapping,
                                    exp)
msfile = SiriusMSFile()
debug_level = 3
sirius_tmp = SiriusTemporaryFileSystemObjects(debug_level)
siriusstring= String(sirius_tmp.getTmpMsFile())
feature_only = sirius_algo.isFeatureOnly()
isotope_pattern_iterations = sirius_algo.getIsotopePatternIterations()
no_mt_info = sirius_algo.isNoMasstraceInfoIsotopePattern()
compound_info = []
msfile.store(exp,
             String(sirius_tmp.getTmpMsFile()),
             feature_mapping, 
             feature_only,
             isotope_pattern_iterations, 
             no_mt_info, 
             compound_info)
out_csifingerid = "./results/Epemicins/interim/structures_Epemicins.mzTab" 
executable= "/path/to/sirius_executable"
subdirs = sirius_algo.callSiriusQProcess(String(sirius_tmp.getTmpMsFile()),
                                         String(sirius_tmp.getTmpOutDir()),
                                         String(executable),
                                         String(out_csifingerid),
                                         False)
candidates = sirius_algo.getNumberOfSiriusCandidates()
sirius_result = MzTab()
siriusfile = MzTabFile()
SiriusMzTabWriter.read(subdirs,
                       filename,
                       candidates,
                       sirius_result)
siriusfile.store("./results/Epemicins/interim/formulas_Epemicins.mzTab", sirius_result) 

In [11]:
siriusfile= "./results/Epemicins/interim/formulas_Epemicins.mzTab"
sirius=  pyteomics.mztab.MzTab(siriusfile, encoding='UTF8', table_format='df')
sirius.metadata
df= sirius.small_molecule_table
SIRIUS_DF= df.drop(columns= ["identifier", "smiles", "inchi_key", "description", "calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
SIRIUS_DF.to_csv("./results/Epemicins/formulas_Epemicins.csv")
SIRIUS_DF= pd.read_csv("./results/Epemicins/formulas_Epemicins.csv", index_col= "Unnamed: 0")
SIRIUS_DF=SIRIUS_DF[SIRIUS_DF["opt_global_explainedIntensity"] >= 0.6] #opt_global_explainedIntensity should be higher than 0.8 or 0.9 even for reliable results
SIRIUS_DF= SIRIUS_DF.sort_values(by= "exp_mass_to_charge")
SIRIUS_DF

,chemical_formula,exp_mass_to_charge,retention_time,best_search_engine_score[1],best_search_engine_score[2],best_search_engine_score[3],opt_global_adduct,opt_gobal_precursorFormula,opt_global_rank,opt_global_explainedPeaks,opt_global_explainedIntensity,opt_global_median_mass_error_fragment_peaks_ppm,opt_global_median_absolute_mass_error_fragment_peaks_ppm,opt_global_mass_error_precursor_ppm,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id
4465,C10H12N3O,208.133234,329.607460,15.230887,14.193324,1.037563,[M + H4 + N]+,C10H16N3O,2,4,0.935599,6.881382,6.881382,6.590543,1576,1577,id_18047075232466844176,controllerType=0 controllerNumber=1 scan=1577|...
4466,C7H16N3OS,208.133234,329.607460,13.604441,13.604441,0.000000,[M + H4 + N]+,C7H20N3OS,3,4,0.935599,6.190684,8.389324,-9.606428,1576,1577,id_18047075232466844176,controllerType=0 controllerNumber=1 scan=1577|...
7143,C9H15N4O2S,261.123425,228.511545,31.635521,31.635521,0.000000,[M + H4 + N]+,C9H19N4O2S,4,13,0.982498,7.138080,7.553053,-7.553053,1057,1058,id_9084564238861654555,controllerType=0 controllerNumber=1 scan=1058
7141,C12H11N4O2,261.123425,228.511545,39.257591,39.257591,0.000000,[M + H4 + N]+,C12H15N4O2,2,14,0.996021,5.349610,5.349610,5.357041,1057,1058,id_9084564238861654555,controllerType=0 controllerNumber=1 scan=1058
7140,C14H13NO3,261.123425,228.511545,46.917919,46.549750,0.368169,[M + H4 + N]+,C14H17NO3,1,11,0.833473,-1.250390,1.250390,0.215089,1057,1058,id_9084564238861654555,controllerType=0 controllerNumber=1 scan=1058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780,C66H120N8O26,1441.841535,354.465876,216.934294,207.480018,9.454275,[M + H]+,C66H120N8O26,7,52,0.891251,-1.461524,1.461524,1.998905,1711,1712,id_5303401137641127839,controllerType=0 controllerNumber=1 scan=1712|...
4781,C65H104N26O12,1441.841535,354.465876,216.806744,207.480018,9.326726,[M + H]+,C65H104N26O12,8,52,0.891251,-1.461524,1.461524,1.078687,1711,1712,id_5303401137641127839,controllerType=0 controllerNumber=1 scan=1712|...
4782,C70H104N24O10,1441.841535,354.465876,216.535272,207.480018,9.055254,[M + H]+,C70H104N24O10,9,52,0.891251,-1.461524,1.461524,-1.711328,1711,1712,id_5303401137641127839,controllerType=0 controllerNumber=1 scan=1712|...
4779,C72H116N10O20,1441.841535,354.465876,216.972120,207.480018,9.492101,[M + H]+,C72H116N10O20,6,52,0.891251,-1.461524,1.461524,-1.718669,1711,1712,id_5303401137641127839,controllerType=0 controllerNumber=1 scan=1712|...


### CSIFinger:ID

The CSI_fingerID function is another algorithm from the Boecher lab, just like SIRIUS adapter and is using the formula predictions from SIRIUS, to search in structural libraries and predict the structure of each formula

In [ ]:
csi_file.store("./results/Epemicins/interim/structures_Epemicins.mzTab", csi_result)
csi_file= "./results/Epemicins/interim/structures_Epemicins.mzTab"
CSI=  pyteomics.mztab.MzTab(csi_file, encoding='UTF8', table_format='df')
CSI.metadata
df= CSI.small_molecule_table
csifingerID= df.drop(columns= ["calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
csifingerID.to_csv("./results/Epemicins/structures_Epemicins.csv)

In [16]:
csifingerID= pd.read_csv("./results/Epemicins/structures_Epemicins.csv",index_col= "Unnamed: 0")
csifingerID

,identifier,chemical_formula,smiles,inchi_key,description,exp_mass_to_charge,retention_time,best_search_engine_score[1],opt_global_rank,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id,opt_global_adduct,opt_global_dblinks,opt_global_dbflags
0,53504|443379|3691183|5362033|5462501|13105154|...,C18H24N2O5,CC(C(=O)N1CCCC1C(=O)O)NC(CCC2=CC=CC=C2)C(=O)O,LZFZMUMEGBBDTC,Enalaprilat,366.202469,294.250456,-279.514811,1,1400,1401,id_4200161612785673241,controllerType=0 controllerNumber=1 scan=1401,[M + H4 + N]+,HMDB:(41886)|ZINC bio:(ZINC01532334 ZINC038128...,17300218222
1,3075197|5862673|7037175|6449058,C18H24N2O5,CC(=O)NC(=CC1=CC=C(C=C1)OC)C(=O)NCCCCCC(=O)O,PUJJFYRIUAKNBK,6-[[(E)-2-acetamido-3-(4-methoxyphenyl)prop-2-...,366.202469,294.250456,-282.923566,2,1400,1401,id_4200161612785673241,controllerType=0 controllerNumber=1 scan=1401,[M + H4 + N]+,COCONUT:(CNP0323497 CNP0390538)|PubChem class ...,70262786
2,73512|290298|36688385|44725560|73900073|739108...,C18H24N2O5,CC(C)C(C(=O)O)NC(=O)C1CCCN1C(=O)OCC2=CC=CC=C2,RMPUPQMKXGKXSG,1-[(benzyloxy)carbonyl]prolylvaline,366.202469,294.250456,-318.076762,3,1400,1401,id_4200161612785673241,controllerType=0 controllerNumber=1 scan=1401,[M + H4 + N]+,NORMAN:(NS00049699)|PubChem:(73512 290298 3668...,270338
3,48427,C18H24N2O5,CCC1(C(=O)NC(=O)N(C1=O)CCC2=C(C=CC(=C2)OC)OC)CC,UAJOMZWNGUKLDS,"5,5-Diethyl-1-(2,5-dimethoxyphenethyl)barbitur...",366.202469,294.250456,-345.891986,4,1400,1401,id_4200161612785673241,controllerType=0 controllerNumber=1 scan=1401,[M + H4 + N]+,PubChem:(48427)|PubChem class - safety and toxic,67108866
4,48426,C18H24N2O5,CCC1(C(=O)NC(=O)N(C1=O)CCC2=C(C=C(C=C2)OC)OC)CC,GUTHIJIXIMLLNU,"5,5-Diethyl-1-(2,4-dimethoxyphenethyl)barbitur...",366.202469,294.250456,-350.990534,5,1400,1401,id_4200161612785673241,controllerType=0 controllerNumber=1 scan=1401,[M + H4 + N]+,PubChem:(48426)|PubChem class - safety and toxic,67108866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,145975,C19H27N3O5S,CC(C)(C(=O)NC(CC1=CC=CC=C1)C(=O)O)NC(=O)C(CCSC...,JCWFZOZLTFSYIA,N-Formyl-met-aib-phe,410.174510,107.215394,-201.763127,1,425,426,id_13266509393966468535,controllerType=0 controllerNumber=1 scan=426,[M + H]+,MeSH:(145975)|PubChem:(145975)|PubMed,70
2579,34464491|85487330,C19H27N3O5S,COC1=CC(=CC(=C1OC)OC)NC(=O)CCCCC2C3C(CS2)NC(=O)N3,IEIFSSZMBSDEJK,NaN,410.174510,107.215394,-234.837636,2,425,426,id_13266509393966468535,controllerType=0 controllerNumber=1 scan=426,[M + H]+,COCONUT:(CNP0358366)|PubChem:(34464491 8548733...,3153922
2580,91638268|110209950|124906482|124906483|124906484,C19H27N3O5S,COC1=CC=C(C=C1)CN2C(=O)C(NC2=O)CCC(=O)NC(CCSC)CO,AOKTYIFAFMAFTA,NaN,410.174510,107.215394,-313.322275,3,425,426,id_13266509393966468535,controllerType=0 controllerNumber=1 scan=426,[M + H]+,PubChem:(91638268 110209950 124906482 12490648...,2097154
2581,91638228|110210331|124862066|124862067|124862068,C19H27N3O5S,COC1=CC=C(C=C1)CCN2C(=O)C(NC2=O)CC(=O)NC(CCSC)CO,LBLQOPATHLEFQU,NaN,410.174510,107.215394,-315.882524,4,425,426,id_13266509393966468535,controllerType=0 controllerNumber=1 scan=426,[M + H]+,PubChem:(91638228 110210331 124862066 12486206...,2097154


#### Explanation of columns
##### mz= mass-to-charge ratio (m/z)
##### RT= retention time (sec)
##### intensity = intensity of the feature (AU-arbitrary units)
##### FWHM= Full Width of the peak at Half its Maximum height
##### num_of_masstraces	= number of mass traces detected (single mass traces are excluded). This is relevant to the isotopic pattern
##### isotope_distances = distance in mz between the isotopes (jumps of app. 1 is important to confirm that this is a real feature) 

### Create a metadata csv file for GNPS from the samples.tsv file 

In [ ]:
import pandas as pd
import numpy as np 
import os

path= "results/GNPSexport/"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir("results/GNPSexport/")
df= pd.read_csv("config/samples.tsv", sep= "\t", header= 0)
metadata= df.rename(columns= {"sample_name": "filename", "comment": "ATTRIBUTE_comment", "MAPnumber": "ATTRIBUTE_MAPnumber"})
metadata["filename"]= metadata["filename"].astype(str) +".mzml"
metadata['ATTRIBUTE_MAPnumber'] = np.arange(len(metadata))
metadata["ATTRIBUTE_comment"]= metadata["ATTRIBUTE_comment"].astype(str) + "_MAP" + metadata["ATTRIBUTE_MAPnumber"].astype(str)
metadata= metadata.drop(columns= "ATTRIBUTE_MAPnumber")
metadata['ATTRIBUTE_genomeID']=metadata['filename'].str.extract(r'(NBC_?\w{5})')
metadata['ATTRIBUTE_genomeIDMDNA']=metadata['filename'].str.extract(r'(MDNA_?\w{5})')
metadata['ATTRIBUTE_genomeID']=metadata['ATTRIBUTE_genomeID'].fillna(metadata['ATTRIBUTE_genomeIDMDNA'])
metadata=metadata.drop(columns="ATTRIBUTE_genomeIDMDNA")
#metadata['ATTRIBUTE_genomeID']= metadata['ATTRIBUTE_genomeID'].replace(to_replace= r'NBC', value= 'NBC_', regex= True)
#metadata['ATTRIBUTE_genomeID']= metadata['ATTRIBUTE_genomeID'].replace(to_replace= r'MDNAWGS', value= 'MDNA_WGS_', regex= True)
metadata.to_csv("results/GNPSexport/metadata.tsv", sep='\t')

From the CLI, I am using the following algorithms to generate files compatible to GNPS for FBMN: 

#### MapAlignerPoseClustering is used to perform a linear retention time alignment, basically correct for linear shifts in retention time. 
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_MapAlignerPoseClustering.html

    rule MapAlignerPoseClustering:
        input:
            expand("results/{samples}/interim/MFD_{samples}.featureXML", samples=SAMPLES)
        output:
            expand("results/Consensus/interim/MapAlignerPoseClustering_{samples}.featureXML", samples=SAMPLES)
        shell:
            """
            resources/OpenMS-2.7.0/bin/MapAlignerPoseClustering -in {input} -out {output}
            """ 

In [ ]:
!resources/OpenMS-2.7.0/bin/MapAlignerPoseClustering -in results/Epemicins/interim/MFD_Epemicins.featureXML results/GermicdinA/interim/MFD_GermicdinA.featureXML results/GermicdinB/interim/MFD_GermicdinB.featureXML -out results/Consensus/interim/MapAlignerPoseClustering_Epemicins.featureXML results/Consensus/interim/MapAlignerPoseClustering_GermicdinA.featureXML results/Consensus/interim/MapAlignerPoseClustering_GermicdinB.featureXML

#### Introduce the features to a protein identification file (idXML)   
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_IDMapper.html

    rule IDMapper:
        input:
            "resources/emptyfile.idXML",
            "results/Consensus/interim/MapAlignerPoseClustering_{samples}.featureXML",
            "results/{samples}/interim/precursorcorrected_{samples}.mzML"
        output:
            "results/Consensus/interim/IDMapper_{samples}.featureXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/IDMapper -id {input[0]} -in {input[1]}  -spectra:in {input[2]} -out {output} 
            """


In [ ]:
!resources/OpenMS-2.7.0/bin/IDMapper -id resources/emptyfile.idXML -in results/GNPSexport/interim/MapAlignerPoseClustering_Epemicins.featureXML  -spectra:in results/Epemicins/interim/precursorcorrected_Epemicins.mzML -out results/GNPSexport/interim/IDMapper_Epemicins.featureXML

#### The FeatureLinkerUnlabeledKD is used to aggregate the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (no MS2 data).
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FeatureLinkerUnlabeledKD.html

    rule FeatureLinkerUnlabeledKD:
        input:
            expand("results/Consensus/interim/IDMapper_{samples}.featureXML", samples=SAMPLES)
        output:
            "results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/FeatureLinkerUnlabeledKD -in {input} -out {output} 
            """

In [ ]:
!resources/OpenMS-2.7.0/bin/FeatureLinkerUnlabeledKD -in results/Consensus/interim/IDMapper_Epemicins.featureXML results/Consensus/interim/IDMapper_GermicidinA.featureXML results/Consensus/interim/IDMapper_GermicidinB.featureXML -out results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML

#### Filter out the features that do not have an MS2 pattern
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FileFilter.html

    rule FileFilter:
        input:
            "results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML"
        output:
            "results/Consensus/filtered.consensusXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/FileFilter -id:remove_unannotated_features -in {input} -out {output} 
            """


In [ ]:
!resources/OpenMS-2.7.0/bin/FileFilter -id:remove_unannotated_features -in results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML -out results/Consensus/filtered.consensusXML


#### GNPS_export creates an mgf file with only the MS2 information of all files (introduce mzml files with spaces between them)
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_GNPSExport.html

    rule GNPS_export:
        input:
            "results/Consensus/filtered.consensusXML",
            expand("results/{samples}/interim/precursorcorrected_{samples}.mzML", samples=SAMPLES)
        output:
            "results/GNPSexport/MSMS.mgf" 
        shell:
            """
            resources/OpenMS-2.7.0/bin/GNPSExport -in_cm {input[0]} -in_mzml {input[1]} -out {output} 
            """

In [8]:
!resources/OpenMS-2.7.0/bin/GNPSExport -in_cm results/Consensus/filtered.consensusXML -in_mzml results/Epemicins/interim/precursorcorrected_Epemicins.mzML results/GermicidinA/interim/precursorcorrected_GermicidinA.mzML results/GermicidinB/interim/precursorcorrected_GermicidinB.mzML -out results/GNPSexport/MSMS.mgf

Progress of 'parsing features and ms2 identifications...':
  1.63 %               GNPSExport took 2.62 s (wall), 2.58 s (CPU), 0.16 s (system), 2.42 s (user); Peak Memory Usage: 87 MB.


#### export the consensusXML file to a txt file for GNPS
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_TextExporter.html
    
    rule txt_export:
        input:
            "results/Consensus/filtered.consensusXML"
        output:
            "results/GNPSexport/FeatureQuantificationTable.txt" 
        shell:
            """
            resources/OpenMS-2.7.0/bin/TextExporter -in {input} -out {output}
            """

In [15]:
!resources/OpenMS-2.7.0/bin/TextExporter -in results/Consensus/filtered.consensusXML -out results/GNPSexport/FeatureQuantificationTable.txt

TextExporter took 0.09 s (wall), 0.09 s (CPU), 0.01 s (system), 0.08 s (user); Peak Memory Usage: 19 MB.


In [21]:
!resources/OpenMS-2.7.0/bin/FeatureFinderMetaboIdent -in results/Epemicins/interim/precursorcorrected_Epemicins.mzML -id MetaboliteIdentification.tsv -out results/Epemicins/FFMI_Epemicins.featureXML

RT window size calculated as 240 seconds.
Loading targets and creating assay library...
')
